
# Анализ A/B-тестирования рекламной кампании

Целью данного исследования является оценка эффективности рекламной кампании и анализ факторов, которые могли повлиять на успех.
Мы будем отвечать на два бизнес-вопроса:

1. Будет ли кампания успешной?
2. Насколько успех кампании можно объяснить именно рекламой?

Данные представлены в файле `marketing_AB.csv`.


In [ ]:

# Импортируем необходимые библиотеки
import pandas as pd

# Загружаем данные
file_path = 'marketing_AB.csv'
data = pd.read_csv(file_path)

# Выводим общую информацию о данных
data.info()

# Просматриваем первые строки датасета
data.head()


In [ ]:

# Удаляем ненужный столбец "Unnamed: 0"
data = data.drop(columns=["Unnamed: 0"])

# Проверяем, есть ли пользователи, попавшие в обе группы
overlap_users = data.groupby("user id")["test group"].nunique()
overlap_users = overlap_users[overlap_users > 1].index

# Удаляем пользователей из обеих групп, если такие есть
cleaned_data = data[~data["user id"].isin(overlap_users)]

# Преобразуем "most ads day" в категориальный тип данных
cleaned_data["most ads day"] = cleaned_data["most ads day"].astype("category")

# Проверяем результат предобработки
cleaned_data.info()



## Выводы по предобработке данных

1. Удален лишний столбец `Unnamed: 0`.
2. Проверено наличие пользователей, попавших в обе группы; таких пользователей не обнаружено.
3. Преобразован тип данных для столбца `most ads day` в категориальный.

Данные готовы для дальнейшего анализа.


In [ ]:

# Расчет основных показателей по группам
group_metrics = cleaned_data.groupby("test group").agg(
    total_visits=("user id", "count"),
    total_conversions=("converted", "sum")
).reset_index()

# Расчет коэффициента конверсии
group_metrics["conversion_rate"] = group_metrics["total_conversions"] / group_metrics["total_visits"]

# Выводим метрики
group_metrics


In [ ]:

# Рассчитываем среднее количество увиденной рекламы
group_ads_metrics = cleaned_data.groupby("test group").agg(
    avg_ads_seen=("total ads", "mean")
).reset_index()

# Объединяем результаты для анализа
group_analysis = group_metrics.merge(group_ads_metrics, on="test group")

# Выводим подробный анализ
group_analysis


In [ ]:

import matplotlib.pyplot as plt

# Строим график конверсий
conversion_data = cleaned_data.groupby("test group")["converted"].mean()
conversion_data.plot(kind="bar", figsize=(8, 6))

# Добавляем подписи
plt.title("Конверсии по группам", fontsize=16)
plt.ylabel("Уровень конверсии", fontsize=14)
plt.xlabel("Группа", fontsize=14)
plt.xticks(rotation=0)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()



## Выводы первичного анализа

1. Уровень конверсии значительно выше в группе "ad" (реклама), чем в группе "psa" (контроль).
2. Среднее количество увиденной рекламы в обеих группах примерно одинаково.
3. Визуализация подтверждает значительную разницу в уровнях конверсии между группами.


In [ ]:

from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import ttest_ind
import numpy as np

# Группы для анализа
ad_group = cleaned_data[cleaned_data["test group"] == "ad"]
psa_group = cleaned_data[cleaned_data["test group"] == "psa"]

# Тест на конверсию
conversion_counts = [ad_group["converted"].sum(), psa_group["converted"].sum()]
conversion_totals = [ad_group["converted"].count(), psa_group["converted"].count()]
z_stat_conversion, p_value_conversion = proportions_ztest(conversion_counts, conversion_totals)

# Тест на количество рекламы
t_stat_ads, p_value_ads = ttest_ind(ad_group["total ads"], psa_group["total ads"], equal_var=False)

# Доверительные интервалы
conversion_rate_diff = ad_group["converted"].mean() - psa_group["converted"].mean()
se_conversion_diff = np.sqrt(
    (ad_group["converted"].mean() * (1 - ad_group["converted"].mean()) / len(ad_group)) +
    (psa_group["converted"].mean() * (1 - psa_group["converted"].mean()) / len(psa_group))
)
ci_conversion_diff = (
    conversion_rate_diff - 1.96 * se_conversion_diff,
    conversion_rate_diff + 1.96 * se_conversion_diff
)

mean_ads_diff = ad_group["total ads"].mean() - psa_group["total ads"].mean()
se_ads_diff = np.sqrt(
    ad_group["total ads"].var() / len(ad_group) +
    psa_group["total ads"].var() / len(psa_group)
)
ci_ads_diff = (
    mean_ads_diff - 1.96 * se_ads_diff,
    mean_ads_diff + 1.96 * se_ads_diff
)

# Результаты
{
    "Conversion Rate Test": {"z_stat": z_stat_conversion, "p_value": p_value_conversion, "ci_diff": ci_conversion_diff},
    "Average Ads Seen Test": {"t_stat": t_stat_ads, "p_value": p_value_ads, "ci_diff": ci_ads_diff}
}



## Выводы статистического анализа

1. **Конверсии:** Разница между уровнями конверсии статистически значима (\( p \)-значение значительно ниже 0.05).
2. **Среднее количество рекламы:** Разница статистически незначима (\( p \)-значение выше 0.05).
3. 95%-ные доверительные интервалы подтверждают эти результаты.

### Общий вывод
Реклама доказала свою эффективность, так как значительно увеличивает уровень конверсии.
